In [1]:
import pandas as pd
import numpy as np
import os
print("import done")
import requests
import json 
import csv 

import done


In [2]:
#getjson data from url and storing it on local system

url = 'https://api.covid19india.org/raw_data.json'

data = requests.get(url=url).json()
data = json.dumps(
    data,
    indent=2,
    ensure_ascii=False,
)

f = open('data/raw.json', 'w')
f.write(data)
f.close()

In [3]:
# JSON file to CSV 
# Opening JSON file and loading the data 
# into the variable data 

with open('data/raw.json') as json_file: 
    data = json.load(json_file) 

patient_data = data['raw_data'] 

# now we will open a file for writing 
data_file = open('data/data_file.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing 
# headers to the CSV file 
count = 0

for pt in patient_data: 
    if count == 0: 

        # Writing headers of CSV file 
        header = pt.keys() 
        csv_writer.writerow(header) 
        count += 1

    # Writing data of CSV file 
    csv_writer.writerow(pt.values()) 

data_file.close() 
print("done")

done


In [7]:
#importing row data 
#This is Indian data for COVID-19 include all infomation of the infected person
raw_data  = pd.read_csv(r"data_file.csv",sep=",",encoding='cp1252')
data= pd.DataFrame(raw_data)
Column_Name=data.columns
Column_Name

#extract usefull column and create a dataframe

Indian_data=data[['patientnumber', 'dateannounced',
        'agebracket', 'gender','detectedstate', 'currentstatus',
       'typeoftransmission','statuschangedate']]
#renaming the column name
Indian_data = Indian_data.rename({"patientnumber": "PatientNumber", "dateannounced" :"InfectionDate",
        "agebracket":"Age","detectedstate":"State", "currentstatus":"Status",
       "typeoftransmission":"TransmissionType","statuschangedate":"StatusChangeDate",'gender':"Gender"}, axis=1)
cols= Indian_data.columns

#removing NA values
Indian_data = Indian_data.loc[:, cols].fillna("-")
Indian_data


,PatientNumber,InfectionDate,Age,Gender,State,Status,TransmissionType,StatusChangeDate
0,1,30/01/2020,20,F,Kerala,Recovered,Imported,14/02/2020
1,2,02/02/2020,-,-,Kerala,Recovered,Imported,14/02/2020
2,3,03/02/2020,-,-,Kerala,Recovered,Imported,14/02/2020
3,4,02/03/2020,45,M,Delhi,Recovered,Imported,15/03/2020
4,5,02/03/2020,24,M,Telangana,Recovered,Imported,02/03/2020
...,...,...,...,...,...,...,...,...
12017,12018,15/04/2020,-,-,Maharashtra,Hospitalized,-,15/04/2020
12018,12019,15/04/2020,-,-,Maharashtra,Hospitalized,-,15/04/2020
12019,12020,15/04/2020,-,-,Maharashtra,Hospitalized,-,15/04/2020
12020,12021,15/04/2020,-,-,Maharashtra,Hospitalized,-,15/04/2020


In [2]:
#Indian_df.index
#Indian_df.describe
